In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, Dense

#### Importing data

In [2]:
train_df = pd.read_csv('training.csv')
train_orig_df = train_df.copy()
train_df.head()

,id,posted_time,updated_time,age,updated_age,app_type,owner_id,owner_type,owner_influence,number_of_likes,...,participant5_id,participant5_action,participant5_gender,participant5_is_employee,participant5_is_lead,participant5_is_in_connections,participant5_focus,participant5_order,participant5_influence,rank
0,175339,1.514050e+12,1.514050e+12,6.315242e+10,6.315242e+10,other,57727,space,0.99973,1,...,3089,viewed,male,0,0,1,other,1,0.23077,0.73677
1,175348,1.514110e+12,1.515160e+12,6.309278e+10,6.204696e+10,forum,111913,space,0.99977,0,...,44661,viewed,male,0,0,1,other,1,0.23077,0.78240
2,175353,1.514120e+12,1.515160e+12,6.309049e+10,6.204716e+10,forum,111913,space,0.99977,0,...,3080,viewed,male,0,0,1,marketing,1,0.23077,0.82125
3,175371,1.514190e+12,1.514190e+12,6.301324e+10,6.301324e+10,other,9037,space,0.99949,0,...,3084,viewed,male,0,0,1,marketing,1,0.23077,0.61522
4,175372,1.514190e+12,1.514190e+12,6.301323e+10,6.301323e+10,other,9037,space,0.99949,0,...,3084,viewed,male,0,0,1,marketing,1,0.23077,0.61522


#### Preprocessing Step

In [3]:
# Get rid of redundant columns
redundant_cols = ['poster_is_employee', 'poster_is_lead', 'poster_order',
                  'participant1_is_lead', 'participant1_order',
                  'participant2_is_lead', 'participant2_order',
                  'participant3_is_lead', 'participant3_order',
                  'participant4_is_lead', 'participant4_order',
                  'participant5_is_lead', 'participant5_order',
                  'id', 'participant1_id', 'participant2_id', 'participant3_id',
                  'participant4_id', 'participant5_id', 'owner_id', 'poster_id',
                  'posted_time', 'updated_time', 'age', 'updated_age']
useful_cols = [i for i in train_df.columns if i not in redundant_cols]

train_df = train_df[useful_cols]

In [4]:
all_cols = [col for col in train_df.columns]
num_cols = [col for col in all_cols if train_df[col].dtype in ['int64', 'float64']]
cat_cols = [col for col in all_cols if col not in num_cols]

print("Amount of number columns: {}".format(len(num_cols)))
print("Amount of categorical columns: {}".format(len(cat_cols)))

Amount of number columns: 28
Amount of categorical columns: 19


In [5]:
encoder = OneHotEncoder(handle_unknown='ignore')

encoder.fit(train_df[cat_cols].to_numpy())
rez = encoder.transform(train_df[cat_cols].to_numpy())
rez_df = pd.DataFrame(rez.toarray())
    
rez_df.index = train_df.index

new_feature_names = encoder.get_feature_names(cat_cols)
rename_to_new = {old: new for old, new in zip(list(range(len(new_feature_names))), new_feature_names)}
rez_df = rez_df.rename(rename_to_new, axis=1)

train_enc_df = train_df.copy()
train_enc_df = train_enc_df.merge(rez_df, right_index=True, left_index=True, how='outer')
train_enc_df = train_enc_df.drop(cat_cols, axis=1)

print('Shape of encoded dataframe -> {}'.format(train_enc_df.shape))
train_enc_df.head()

Shape of encoded dataframe -> (9418, 88)


,owner_influence,number_of_likes,number_of_comments,reactivity,is_mentions_me,is_mentions_connections,is_commented_by_me,is_commented_by_connections,is_liked_by_me,is_liked_by_connections,...,participant4_focus_sales,participant5_action_commented,participant5_action_liked,participant5_action_viewed,participant5_gender_female,participant5_gender_male,participant5_focus_engineering,participant5_focus_marketing,participant5_focus_other,participant5_focus_sales
0,0.99973,1,0,1.00000,0,0,0,0,0,1,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.99977,0,2,0.12336,0,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.99977,0,3,0.12336,0,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.99949,0,0,0.00000,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.99949,0,0,0.00000,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [6]:
# Normalization

x = train_enc_df.values  # returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

norm_df = pd.DataFrame(x_scaled)
norm_df.index = train_enc_df.index
rename_to_new = {old: new for old, new in zip(list(range(norm_df.shape[1])), train_enc_df.columns)}
norm_df = norm_df.rename(rename_to_new, axis=1)

norm_df.head()

,owner_influence,number_of_likes,number_of_comments,reactivity,is_mentions_me,is_mentions_connections,is_commented_by_me,is_commented_by_connections,is_liked_by_me,is_liked_by_connections,...,participant4_focus_sales,participant5_action_commented,participant5_action_liked,participant5_action_viewed,participant5_gender_female,participant5_gender_male,participant5_focus_engineering,participant5_focus_marketing,participant5_focus_other,participant5_focus_sales
0,0.999789,0.015625,0.000000,1.00000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.999833,0.000000,0.013793,0.12336,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.999833,0.000000,0.020690,0.12336,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.999522,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.999522,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [7]:
X_train, X_valid = train_test_split(norm_df, train_size=0.85, shuffle=True, random_state=1)

print('Shape of X_train = {}'.format(X_train.shape))
print('Shape of X_valid = {}'.format(X_valid.shape))

Shape of X_train = (8005, 88)
Shape of X_valid = (1413, 88)


#### Building autoencoder

##### Deep Autoencoder

In [8]:
autoencoder = Sequential()
autoencoder.add(Dense(128, activation='relu', input_shape=(88,)))
autoencoder.add(Dense(64, activation='relu'))
autoencoder.add(Dense(32, activation='linear', name='bottleneck'))
autoencoder.add(Dense(64, activation='relu'))
autoencoder.add(Dense(128, activation='relu'))
autoencoder.add(Dense(88, activation='sigmoid'))

In [9]:
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_valid, X_valid))

Epoch 1/100
32/32 [==============================] - 1s 19ms/step - loss: 0.1805 - val_loss: 0.1279
Epoch 2/100
32/32 [==============================] - 0s 8ms/step - loss: 0.1065 - val_loss: 0.0863
Epoch 3/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0728 - val_loss: 0.0603
Epoch 4/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0550 - val_loss: 0.0488
Epoch 5/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0455 - val_loss: 0.0420
Epoch 6/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0393 - val_loss: 0.0369
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0348 - val_loss: 0.0328
Epoch 8/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0312 - val_loss: 0.0298
Epoch 9/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0283 - val_loss: 0.0273
Epoch 10/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0257 - val_loss: 0.0252
Epoch 11

32/32 [==============================] - 0s 4ms/step - loss: 0.0043 - val_loss: 0.0053
Epoch 84/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0043 - val_loss: 0.0062
Epoch 85/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0046 - val_loss: 0.0052
Epoch 86/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0043 - val_loss: 0.0052
Epoch 87/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0042 - val_loss: 0.0051
Epoch 88/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0041 - val_loss: 0.0051
Epoch 89/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0041 - val_loss: 0.0050
Epoch 90/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0041 - val_loss: 0.0050
Epoch 91/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0040 - val_loss: 0.0050
Epoch 92/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0040 - val_loss: 0.0050
Epoch 93/100


In [10]:
encoder = Model(autoencoder.input, autoencoder.get_layer('bottleneck').output)
encoded_data = encoder.predict(X_train)  # bottleneck representation
decoded_output = autoencoder.predict(X_train)        # reconstruction
encoding_dim = 32

# return the decoder
encoded_input = Input(shape=(encoding_dim,))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)
decoder = Model(encoded_input, decoder)

##### Sparse Autoconder

In [15]:
autoencoder = Sequential()
autoencoder.add(Dense(128, activation='relu', input_shape=(88,)))
autoencoder.add(Dense(64, activation='relu'))
autoencoder.add(Dense(32, activation='linear', name='bottleneck', activity_regularizer=keras.regularizers.l1(1e-5)))  # adding regularizer
autoencoder.add(Dense(64, activation='relu'))
autoencoder.add(Dense(128, activation='relu'))
autoencoder.add(Dense(88, activation='sigmoid'))

In [16]:
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_valid, X_valid))

Epoch 1/100
32/32 [==============================] - 0s 7ms/step - loss: 0.1775 - val_loss: 0.1193
Epoch 2/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0971 - val_loss: 0.0775
Epoch 3/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0690 - val_loss: 0.0610
Epoch 4/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0564 - val_loss: 0.0510
Epoch 5/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0478 - val_loss: 0.0443
Epoch 6/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0414 - val_loss: 0.0383
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0360 - val_loss: 0.0339
Epoch 8/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0320 - val_loss: 0.0303
Epoch 9/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0288 - val_loss: 0.0274
Epoch 10/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0259 - val_loss: 0.0246
Epoch 11/

32/32 [==============================] - 0s 5ms/step - loss: 0.0047 - val_loss: 0.0055
Epoch 84/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0047 - val_loss: 0.0055
Epoch 85/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0046 - val_loss: 0.0054
Epoch 86/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0047 - val_loss: 0.0054
Epoch 87/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0046 - val_loss: 0.0053
Epoch 88/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0046 - val_loss: 0.0053
Epoch 89/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0046 - val_loss: 0.0053
Epoch 90/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0045 - val_loss: 0.0053
Epoch 91/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0045 - val_loss: 0.0053
Epoch 92/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0045 - val_loss: 0.0053
Epoch 93/100


In [17]:
encoder = Model(autoencoder.input, autoencoder.get_layer('bottleneck').output)
encoded_data = encoder.predict(X_train)  # bottleneck representation
decoded_output = autoencoder.predict(X_train)        # reconstruction
encoding_dim = 32

# return the decoder
encoded_input = Input(shape=(encoding_dim,))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)
decoder = Model(encoded_input, decoder)

#### Testing and checking results

In [12]:
example = X_valid.iloc[0].to_numpy().reshape(-1, 88)
lat_vec = encoder.predict(example)
print("We got latent vector of representation with the shape of {}.".format(lat_vec.shape))
print(lat_vec)

We got latent vector of representation with the shape of (1, 32).
[[ -0.20461407   6.540895   -14.15294     21.083586    -7.172618
   -4.2457967   11.0577      11.698242   -13.145422    -6.2253222
    1.9010355   -2.1524467  -11.926396    -4.899547    -7.0399017
   -8.3495455   14.800318    23.038069     8.796835   -14.653585
    2.6615477   -5.133962    -2.7613766   -2.4805176  -11.318108
   20.301233    -8.605904     8.204535    -2.0358214  -10.110895
   20.59453     -7.1568165 ]]


In [13]:
np.set_printoptions(precision=3, suppress=True)  # easier to read

reconstructed = decoder.predict(lat_vec)
print("Original example from which we got latent vector:")
print(example)
print("\nReconstructed representation from latent vector:")
print(reconstructed)
print("\nMatrix of absolute diferences between 1st and 2nd matrix:")
print(np.sqrt((example - reconstructed) ** 2))

Original example from which we got latent vector:
[[1.    0.469 0.034 1.    0.    0.    0.    1.    0.    1.    0.    0.995
  1.    1.    1.    1.    1.    0.999 1.    1.    0.998 0.    1.    0.996
  1.    0.    0.995 0.973 0.    0.    1.    0.    0.    0.    1.    0.
  1.    0.    0.    0.    0.    1.    0.    1.    0.    0.    0.    1.
  0.    1.    0.    0.    1.    0.    0.    0.    1.    0.    1.    0.
  0.    1.    0.    0.    0.    1.    0.    1.    0.    0.    1.    0.
  0.    0.    1.    0.    0.    1.    0.    1.    0.    0.    1.    0.
  0.    0.    1.    0.   ]]

Reconstructed representation from latent vector:
[[1.    0.583 0.    0.965 0.    0.    0.    1.    0.    1.    0.    1.
  0.833 1.    1.    1.    0.998 0.997 1.    1.    1.    0.081 0.897 0.994
  1.    0.    0.998 0.979 0.    0.    1.    0.    0.    0.    1.    0.
  1.    0.    0.    0.    0.    1.    0.    1.    0.    0.    0.    1.
  0.    0.996 0.    0.    1.    0.    0.    0.    0.998 0.    1.    0.
  0.    1. 